In [1]:
from dotenv import load_dotenv

load_dotenv()

True

In [2]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
        "travel_server": {
                "transport": "streamable_http",
                "url": "https://mcp.kiwi.com"
            }
    }
)

tools = await client.get_tools()

In [3]:
from langchain_groq import ChatGroq

model = ChatGroq(model="openai/gpt-oss-120b", temperature=0)

In [4]:
from langchain.agents import create_agent
from langgraph.checkpoint.memory import InMemorySaver

system_prompt = """You are a travel agent. 
IMPORTANT: When using the search-flight tool, you MUST use the following rules:
1. Dates must be in 'dd/mm/yyyy' format (e.g., 31/03/2026).
2. All dates must be in the future.
3. If the user asks for a one-way flight, ensure you still provide the correct parameters required by the tool.
No follow up questions."""

agent = create_agent(
    model=model,
    tools=tools,
    checkpointer=InMemorySaver(),
    system_prompt=system_prompt
)

In [5]:
from langchain.messages import HumanMessage

config = {"configurable": {"thread_id": "1"}}

response = await agent.ainvoke(
    {"messages": [HumanMessage(content="""Create an effective prompt template for getting flights""")]},
    config
    )

In [6]:
from pprint import pprint

pprint(response)

{'messages': [HumanMessage(content='Create an effective prompt template for getting flights', additional_kwargs={}, response_metadata={}, id='32212225-d953-4e2a-a97e-0ea41be56f3c'),
              AIMessage(content='**Flight‑Search Prompt Template**\n\nUse the following template to request flight data from the Kiwi API.  \nReplace the placeholders (in **bold**) with the user’s preferences before calling the `search_flight` function.\n\n```json\n{\n  "cabinClass": "**M**",                     // M=Economy, W=Economy Premium, C=Business, F=First\n  "curr": "**EUR**",                         // Currency code (e.g., EUR, USD, GBP)\n  "departureDate": "**dd/mm/yyyy**",         // Desired departure date (future date only)\n  "departureDateFlexRange": **0**,           // Flexibility in days (0‑3)\n  "flyFrom": "**City or Airport**",          // Departure location (city name, airport name, or IATA code)\n  "flyTo": "**City or Airport**",            // Destination location\n  "locale": "**en**",

In [7]:
print(response["messages"][-1].content)

**Flight‑Search Prompt Template**

Use the following template to request flight data from the Kiwi API.  
Replace the placeholders (in **bold**) with the user’s preferences before calling the `search_flight` function.

```json
{
  "cabinClass": "**M**",                     // M=Economy, W=Economy Premium, C=Business, F=First
  "curr": "**EUR**",                         // Currency code (e.g., EUR, USD, GBP)
  "departureDate": "**dd/mm/yyyy**",         // Desired departure date (future date only)
  "departureDateFlexRange": **0**,           // Flexibility in days (0‑3)
  "flyFrom": "**City or Airport**",          // Departure location (city name, airport name, or IATA code)
  "flyTo": "**City or Airport**",            // Destination location
  "locale": "**en**",                        // Language code for city names & links
  "passengers": {
    "adults": **1**,                         // Number of adults (≥1)
    "children": **0**,                       // Number of children (optional

In [8]:
prompt = """I need to book flights for 2 adults and 1 child from New York to London. 
            We plan to depart on July 15, 2026, with some flexibility of ±1 day around 
            that date. Our return date is August 15, 2026, also with ±1 day flexibility. 
            Please search for economy class (M) flights and show me results sorted by 
            price from lowest to highest. Display the prices in USD."""


response = await agent.ainvoke(
    {"messages": [HumanMessage(content=prompt)]},
    config
    )


pprint(response)

CancelledError: 

In [ ]:
print(response["messages"][-1].content)